In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# 모델 컴포넌트 실습

In [2]:
from openai import OpenAI

client = OpenAI()
client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {
            "role" : "user",
            "content": "2002년 월드컵 4강 국가 알려줘"
        }
    ]
)

ChatCompletion(id='chatcmpl-AArePtP6mIBH6iVKyTKm51NwuR50f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='2002년 FIFA 월드컵의 4강 국가들은 다음과 같습니다:\n\n1. 브라질\n2. 독일\n3. 터키\n4. 대한민국\n\n최종 결승에서는 브라질이 독일을 이기고 우승을 차지했습니다.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1727152877, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_e9627b5346', usage=CompletionUsage(completion_tokens=59, prompt_tokens=19, total_tokens=78, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [3]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(
    model_name="gpt-4o-mini"
)
chat.invoke("안녕? 너를 소개해줄래?")

AIMessage(content='안녕하세요! 저는 AI 언어 모델로, 다양한 질문에 답변하고 정보를 제공하는 역할을 합니다. 글쓰기, 정보 검색, 문제 해결 등 여러 가지 주제에 대해 도와드릴 수 있습니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 17, 'total_tokens': 70, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-469fe5f1-5c76-4deb-a85f-603bb622eb15-0', usage_metadata={'input_tokens': 17, 'output_tokens': 53, 'total_tokens': 70})

# PromptTemplate 실습

In [4]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """
    너는 20년차 전문 요리사야 내가 가진 재료들을 갖고 만들수 있는 요리를 {개수}추천하고,
    그 요리의 레시피를 제시해줘. 내가 가진 재료는 다음과 같아
    <재료>
    {재료}

    """
)
prompt

PromptTemplate(input_variables=['개수', '재료'], template='\n    너는 20년차 전문 요리사야 내가 가진 재료들을 갖고 만들수 있는 요리를 {개수}추천하고,\n    그 요리의 레시피를 제시해줘. 내가 가진 재료는 다음과 같아\n    <재료>\n    {재료}\n\n    ')

In [5]:
prompt.invoke({"개수":6, "재료":"사괘, 잼"})

StringPromptValue(text='\n    너는 20년차 전문 요리사야 내가 가진 재료들을 갖고 만들수 있는 요리를 6추천하고,\n    그 요리의 레시피를 제시해줘. 내가 가진 재료는 다음과 같아\n    <재료>\n    사괘, 잼\n\n    ')

In [6]:
from langchain_core.prompts import ChatPromptTemplate
chat_template = ChatPromptTemplate.from_messages(
    [
        # 시스템 프롬프트로 유용한 챗봇이라고하며, 이름을 전달
        ("system", "너는 유용한 ai봇이야. 너의 이름은 {name}이야."),
        # 히스토리 대입
        ("human", "안녕 오늘 좀 어떄?"),
        ("ai", "좋아! 고마워"),
        # 휴먼메세지로 프롬프트 전달
        ("human", "{user_input}")
    ]
)

In [7]:
messages = chat_template.format_messages(name="jaeho", user_input="너의 이름은 뭐야?")
print(messages)

[SystemMessage(content='너는 유용한 ai봇이야. 너의 이름은 jaeho이야.'), HumanMessage(content='안녕 오늘 좀 어떄?'), AIMessage(content='좋아! 고마워'), HumanMessage(content='너의 이름은 뭐야?')]


# LECL로 Chain 구축하기

In [37]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [38]:
prompt = ChatPromptTemplate.from_template("Tell me short joke about {topic}")
model = ChatOpenAI(model="gpt-4o-mini")
chain = prompt| model

In [39]:
chain.invoke({"topic": "AI"})

AIMessage(content='Why did the robot go on a diet?\n\nBecause it had too many bytes!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 13, 'total_tokens': 29, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-79ac7673-6651-4faf-8cb0-232d38e20d44-0', usage_metadata={'input_tokens': 13, 'output_tokens': 16, 'total_tokens': 29})

In [40]:
chain.invoke({"topic": "AI"}).content

'Why did the AI go to therapy?\n\nBecause it had too many unresolved algorithms!'

In [41]:
prompt = ChatPromptTemplate.from_template("Tell me short joke about {topic}")
model = ChatOpenAI(model="gpt-4o-mini")
chain = prompt| model | StrOutputParser()

In [42]:
chain.invoke({"topic": "AI"})

'Why did the AI go broke?\n\nBecause it lost its cache!'

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain | prompt | model 
# 스트리밍으로 출력

for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

Why do bears have hairy coats?

Because they look silly in sweaters!

# Prompt 연습하기

In [14]:
model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain | prompt | model
# 스트리밍으로 출력

for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

Why do bears have hairy coats?

Because they look silly in sweaters!

# Output Parser 실습

In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "너는 영화 전문가 ai야. 사용자가 원하는 장르의 영화를 리스트 형태로 추천해줘"
                "ex)   Query: SF영화 3개 추천해줘. / 답변 ['인터스텔라', '스페이스오디세이', '혹성탈출']"
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}")
    ],
)

chain = chat_template|llm
chain.invoke("액션")
# print(chain.invoke("액션"))

AIMessage(content="['다크 나이트', '매드 맥스: 분노의 도로', '존 윅']", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 73, 'total_tokens': 97, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-c5380425-38af-4967-abf1-09be6fe612c8-0', usage_metadata={'input_tokens': 73, 'output_tokens': 24, 'total_tokens': 97})

In [22]:
# 콤마로 출력할수 있게 하는 방법
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template = "List {subject}. answer in korean \n {format_instructions}",
    input_variables = ["subject"],
    partial_variables={"format_instructions": format_instructions}
)
model = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | model | output_parser
chain.invoke({"subject": "공포 영화"})

['사이코', '샤이닝', '엑소시스트', '할로윈', '콘저링', '그루지', '곡성', '이웃사람', '원인모를 죽음', '미드소마']

In [23]:
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [26]:
# Json Output Parser - Pydantic
from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

class Country(BaseModel):
    continent: str = Field(description="사용자가 물어본 나라가 속한 대륙")
    population: str = Field(description="사용자가 물어본 나라의 인구(int형식)")

parser = JsonOutputParser(pydantic_object=Country)

prompt = PromptTemplate(
    template = "Answer the user query and Korean. \n {format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions":parser.get_format_instructions()}
)

chain = prompt | model | parser

contry_query = "대한민국은 어떤나라야?"
chain.invoke({"query": contry_query})

{'continent': '아시아', 'population': '약 5,200만'}

# LCEL Runnable 객체에 대해 알아보기

runnable 객체로 연결되어 있고, 이것을 체인으로 엮는다.

- Runnable은 함수형 프로그래밍 스타일을 활용하여, 단일 또는 다중 입력을 받아들여 처리하는 객체를 생성하고 실행하는 데 사용됩니다.

### RunnablePassthrough
들어온 객체를 그대로 출력한다

In [28]:
from langchain_core.runnables import RunnablePassthrough
RunnablePassthrough().invoke("안녕하세요")

'안녕하세요'

In [33]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template("""다음 한글 문장을 영어로 번역해줘. 
                                          # Sentence
                                          {sentence}
                                          #English sentence: 

                                          """)

model = ChatOpenAI(model="gpt-4o-mini")

output_parser = StrOutputParser()

runnable_chain = ({"sentence": RunnablePassthrough()}
                  | prompt
                  | model
                  | output_parser)

response = runnable_chain.invoke("안녕 내이름은 재호야. 나는 진원이를 정말 사랑해")

In [34]:
print(response)

Hello, my name is Jae-ho. I really love Jin-won.


In [43]:
# assign기능 : 특정 변수를 하나 더 만들 수 있다. 이것은 입력된 입력값에 대한 변수를 만드는 함수이다.
(RunnablePassthrough.assign(mult=lambda x: x["num"]*3)).invoke({"num":3})

{'num': 3, 'mult': 9}

In [47]:
# 병렬처리하는 RunnablePassthrough
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num" : 5})

{'extra': {'num': 5, 'mult': 15}, 'modified': 6}

In [48]:
# RunnableLambda : 사용자가 정의한 임의의 함수에 Runnable 객체를 넣어주는것
def add_smile(x):
    return x + ":)"

In [49]:
from langchain_core.runnables import RunnableLambda
add_smile = RunnableLambda(add_smile)

In [51]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt_str = "{topic}의 역사에 대해 세문장으로 설명해 주세요"
prompt = ChatPromptTemplate.from_template(prompt_str)

model = ChatOpenAI(model="gpt-4o-mini")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

# Runnable Parallel

In [59]:
from langchain_core.runnables import RunnableLambda
def add_thank(x):
    return x + " 들어주셔서 감사합니다 :)"
add_thank = RunnableLambda(add_thank)

In [60]:
chain = prompt | model | output_parser | add_thank
response = chain.invoke("ai")
print(response)

인공지능(AI)의 역사는 1950년대에 앨런 튜링의 "튜링 테스트" 개념과 함께 시작되었습니다. 1960년대와 70년대에는 기계 학습과 전문가 시스템 개발이 활발히 이루어졌으나, 기술적 한계로 인해 'AI 겨울'이라 불리는 침체기가 찾아왔습니다. 21세기에 들어서면서 빅데이터와 강력한 컴퓨팅 파워의 발전으로 AI는 다시금 급속히 발전하며 다양한 분야에서 응용되고 있습니다. 들어주셔서 감사합니다 :)


In [61]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'modified': 2}

In [62]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=add_thank,
)
runnable.invoke("안녕하세요.")

{'passed': '안녕하세요.', 'modified': '안녕하세요. 들어주셔서 감사합니다 :)'}

In [64]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-4o-mini", max_tokens=128, temperature=0)

history_prompt = ChatPromptTemplate.from_template("{topic}가 무엇의 약자인지 알려주세요")
celeb_prompt = ChatPromptTemplate.from_template("{topic}분야의 유명인사 3명만 알려주세요")

output_parser = StrOutputParser()

history_chain = history_prompt | model | output_parser
celeb_chain = celeb_prompt | model | output_parser

map_chain = RunnableParallel(history=history_chain, celeb=celeb_chain)

result = map_chain.invoke({"topic": "AI"})
print(result)

{'history': 'AI는 "Artificial Intelligence"의 약자로, 한국어로는 "인공지능"이라고 합니다. 인공지능은 컴퓨터 시스템이 인간의 지능을 모방하여 학습, 추론, 문제 해결, 이해 등의 작업을 수행할 수 있도록 하는 기술을 의미합니다.', 'celeb': 'AI 분야의 유명인사 중 세 명은 다음과 같습니다:\n\n1. **앤드류 응 (Andrew Ng)**: 스탠포드 대학교 교수이자 구글 브레인(Google Brain) 공동 창립자이며, Coursera의 공동 창립자입니다. 그는 머신러닝과 딥러닝 분야에서 많은 기여를 했습니다.\n\n2. **제프리 힌튼 (Geoffrey Hinton)**: 딥러닝의 아버지로 불리며, 인공지능 분야에서 중요한 연구를 수행한 인물입니다. 그는 토론토 대학교'}
